# Creating a cloud mask for Sentinel-2 imagery

To continue working on our final project, I wanted to further play around with filtering satellite images by cloud coverage, since my attempt in a3 was not successful.
In order to do so I followed the information given in the GEE documentation ([Sentinel-2 Cloud masking with s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless)). I hope that this will provide a solid basis to work on for ourt project, since we have the idea to first roughly filter the collection (with a low high threshold of cloudless imagery), to then continue with only filtering the area of interest, which may be done on the pixel level.

The task given requires us to use imports and ccreate docstrings, which is why I only showed the docstring in this notebook, and the resulting map in the end. The file `a4.py` contains additional comments on the procedure done.

I chose to import everything from the `a4.py`-script, to not have an import for every function, this is not advisable for larger programs as it may negatively impact performance.

I decided to create an image collection for the area surrounding Darmstadt, I looked for a fitting time frame and the coordinates needed in the earth engine app.

##Getting started

I started by importing the earth engine module provided by the python GEE API and also Folium, to later be able to add and display the masked image on a map. I then authenticated GEE and initialized the library i wanted to use.

Then I set the following parameters:

`AOI = ee.Geometry.Point(8.642, 49.877)` - set area of interest (Darmstadt)

`START_DATE = "2024-06-03"` - set start date for collection

`END_DATE = "2024-06-04"` - set end date for point up until which images need to be included

`CLOUD_FILTER = 60` - set threshold on how many clouds are allowed (60%), the threshold is quite high to be able to showcase the masking better, and to inlude more images in the collection (%)

`CLD_PRB_THRESH = 50` - set cloud probability, everything with a greater probability value in the collection is considered cloud (%)

`NIR_DRK_THRESH = 0.15` - set Near Infrared reflectance for dark pixels that may be cloud shadows

`CLD_PRJ_DIST = 1` - maximum distance from cloud to search for shadows (km)

`BUFFER = 50` - buffer distance of cloud objects to include cloud-edges(m)

## Join Sentinel 2 image collection with cloud probability collection


In [17]:
from a4 import *

help(get_s2_sr_cld_col)

Help on function get_s2_sr_cld_col in module a4:

get_s2_sr_cld_col(aoi, start_date, end_date)
    Retrieves filtered collections of sentinel 2 images and joins the collection with
    a cloud propability dataset to better detect clouds

    Parameters:
        aoi - ee.Geometry : Coordinates for area of interest
        start_date - str : start date for collection (inclusive)
        end_date - str : end date for colection (exclusive)

    Returns:
        ee.ImageCollection: filtered collection of Sentinel-2 images joined with cloud
        probability data



## Add Clouds to mask


In [18]:
help(add_cloud_bands)

Help on function add_cloud_bands in module a4:

add_cloud_bands(img)
    Adds data as bands from previous dataset to the image

    Parameters:
        img - ee.Image : input image that will be mask

    Returns:
        img / ee.Image : original image, with bands added



## Identify potential shadows

In order to identify the shadows cast by the clouds, first we select all the values from the Scene Classification Values (SCL) - Band, that are not classified as water.

Then we proceed by identifying dark pixels and calculating the mean angle from which the clouds cast shadows, to then classify the shadows.


In [19]:
help(add_shadow_bands)

Help on function add_shadow_bands in module a4:

add_shadow_bands(img)
    Identifies potential cloud shadows by pxel value and adds them to mask

    Parameters:
        img - ee.Image : input image, taht will be mask

    Returns:
        ee.Image : mask, now with added shadow and dark px bands



In [20]:
help(add_cld_shdw_mask)

Help on function add_cld_shdw_mask in module a4:

add_cld_shdw_mask(img)
    Combines cloud and shadow identification from separate functions into a single cloud-
    shadow mask. This mask helps in visualizing and analyzing areas affected by clouds and
    shadows.

    Parameters:
    - img (ee.Image): The input image to which cloud-shadow mask will be added.

    Returns:
    - ee.Image: The original image with added cloud-shadow mask band.



In [21]:
help(add_ee_layer)

Help on function add_ee_layer in module a4:

add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0)
    Adds an Earth Engine image layer to a Folium map. This function facilitates the
    visualization of Earth Engine datasets on interactive maps.

    Parameters:
    - self (folium.Map): The Folium map instance to which the layer will be added.
    - ee_image_object (ee.Image): The Earth Engine image object to visualize.
    - vis_params (dict): Visualization parameters including bands, min/max values, and gamma.
    - name (str): The name of the layer.
    - show (bool): Whether to display the layer initially.
    - opacity (float): Opacity level of the layer.
    - min_zoom (int): Minimum zoom level at which the layer should be visible.



In [22]:
help(display_cloud_layers)

Help on function display_cloud_layers in module a4:

display_cloud_layers(col)
    Displays various cloud-related layers derived from a Sentinel-2 image collection on a
    Folium map. This includes cloud probability, cloud masks, and shadow projections.

    Parameters:
    - col (ee.ImageCollection): The Sentinel-2 image collection to process and display.



In [23]:
#from a4 import display_cloud_layers,s2_sr_cld_col_eval_disp 

display_cloud_layers(s2_sr_cld_col_eval_disp)
